In [1]:
#importing used packages
import csv
from selenium import webdriver 
import pandas as pd
import numpy as np
import time
import requests
import bs4
import re

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

# Function to get number total page number

In [3]:
def function_to_get_page_number(driver,product_name):
    
    driver.get("https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1".format(product_name.lower()))
    time.sleep(0.1)
    
    page = driver.find_elements_by_xpath('//div[@class="_2MImiq"]/span[1]')
    
    for x in page:
        a = x.text.split()
    
    return int(a[-1])

# Function to get the Products link

In [4]:
def function_to_get_product_links(product_name):
    
    driver = webdriver.Chrome(PATH)
    
    #extract number of pages
    page = function_to_get_page_number(driver,product_name)
    
    #link has no spaces so we are changing sapce with "+"
    product_name = re.subn(" ","+",product_name)[0]    
    print("Number of pages : ",page)
    links = []
    
    #extract all the product links from every page
    for i in range(1,page+1):
        driver.get("https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=".format(product_name.lower())+str(i))
        time.sleep(0.1)
        
        #flipkart has 2 view style list and grid in each style product has different class 
        #here 1 xpath is of list style and 2nd is of grid style
        link =  driver.find_elements_by_xpath('//a[@class="_1fQZEK"]') or driver.find_elements_by_xpath('//a[@class="s1Q9rs"]')
        for x in link:
            links.append(x.get_attribute("href"))
            
    driver.quit()
    print("Number of Products : ", len(links))
    
    return links

# Function to get product features

In [5]:
def function_to_get_featues(url):
    try:
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        col =["Name","Price"]
        while(True):
            table = soup.find("table", class_="_14cfVK")
            
            #break the loop when no more table found
            if table is None:
                break
            for row in table.findAll("tr"):
                cols = row.find_all("td")
                k = cols[0].text.strip()
                col.append(k)
            
            table.decompose()       ##drop the current table 
        
    except:
        print("\nFeatures are : ",col)
    return col

# Function to get data from flipkart

In [6]:
def get_data_from_flipkart():
    
    #get Product name
    product_name = input("Enter the product name: ")
    
    #extract all link
    link = function_to_get_product_links(product_name)
    
    a = np.arange(len(link))
    
    #extract all feature
    col = function_to_get_featues(link[0])
    
    #create DataFrame with extracted feature and rows equal to number of link
    product = pd.DataFrame(index = a,columns=col)
    
    for i in a:
        try:
            res = requests.get(link[i])
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text, "html.parser")
            
            #extract name of the product
            name = soup.find(class_='B_NuCI')
            n = name.text.strip()
            product.loc[i,'Name'] = n
            
            #extarct price of the product
            price = soup.find(class_='_30jeq3 _16Jk6d')
            p = price.text.strip()
            product.loc[i,'Price'] = p[1:]
            
            #there are multiple table of same class from which data was  extracted
            #for this we first selected a table extracted the data and droped it then selected the next table
            while(True):
                table = soup.find("table", class_="_14cfVK")
                
                #break the loop when no more table found
                if table is None:
                    break
                row_data = []
                #key = []
                #value = []
                for row in table.findAll("tr"):
                    cols = row.find_all("td")
                    k = cols[0].text.strip()
                    
                    
                    #unable the next line if you want to keep all feature you get
                    #col.append(k)
                    
                    if k in col:
                        for ele in cols[1:]:
                                v = ele.text.strip()
                        #value.append(v)
                        #key.append(k)
                        product.loc[i,k] = v
                
                #code to see
                #row_data = { k:v for (k,v) in zip(key, value)}  
                #print("\n",row_data)
                
                table.decompose()       ##drop the current table
        except:
            print("\n\n Finish \n\n")
            
    #to save csv file same as product name
    product.to_csv(product_name+'.csv')
    return product

#key and value list is made to see all the data we are getting 

In [7]:
product = get_data_from_flipkart()
product

Enter the product name: wifi router
Number of pages :  20
Number of Products :  767


,Name,Price,Type,Model,In The Box,Color,Brand,Frequency,Frequency Band,Wireless Speed,...,Encryption,Other Security Features,Firewall,Other Features,Certification,VPN,LED Indicator,Installation Features,Series,Power Consumption
0,TP-link TL-WR902AC 750 Mbps Wireless Travel Ro...,"1,699",Wireless Without Modem,TL-WR902AC,"1 Router, Power Adapter, RJ45 Ethernet Cable, ...",White,TP-Link,"2.4 GHz, 5 GHz",Dual Band,750 Mbps,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TP-Link TL-WR940N Wireless N 450 Mbps Wireless...,"1,209",Wireless Without Modem,TL-WR940N Wireless N,"Router, Adaptor, Lan Cord",Black,TP-Link,2.4 GHz,Single Band,450 Mbps,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TP-Link Archer C20 750 MBPS Wireless Router (...,"1,399",Wireless Without Modem,Archer C20,"Wi-Fi Router Archer C20, Power Adapter, RJ45 E...",Blue,TP-Link,"2.4 GHz, 5 GHz",Dual Band,750 MBPS,...,"64/128-bit WEP, WPA/WPA2, WPA-PSK/WPA2-PSK","Quality of Service: WMM, Bandwidth Control, Pa...","DoS, SPI Firewall, IP Address Filtering / MAC ...","Management: Remote Management, Access Control,...","CE, FCC, RoHS","VPN Pass-Through: PPTP, L2TP, IPSec, Port Forw...",NaN,NaN,NaN,NaN
3,TP-Link TL-WR845N Wireless N 300 mbps Wireless...,999,Wireless Without Modem,TL-WR845N Wireless N,"Wireless N Rotuer TL-WR845N, Power Supply Unit...",White,TP-Link,2.4 Ghz,Single Band,300 mbps,...,"64/128/152-bit WEP / WPA / WPA2, WPA-PSK / WPA...","Parental Control, Local Management Control, Ho...","DoS, SPI Firewall, IP Address Filter / MAC Add...","Guest Network: 2.4 GHz Guest Network x 1, IP B...","CE, FCC, RoHS",NaN,Yes,NaN,NaN,NaN
4,D-Link DIR-650IN 300 Mbps Wireless Router (Bl...,899,Wireless Without Modem,DIR-650IN,"1 x Router, 1 x Power Adapter, 1 x Ethernet Ca...",Black,D-Link,2.4 GHz,Single Band,300 Mbps,...,"WPA, WPA2, WEP 64/128","IWPA & WPA2 (Wi-Fi Protected Access, Wi-Fi Pro...",NaN,Multiple Operating Modes: Router | AP | Repeat...,NaN,NaN,"Power, Internet, LAN, Wi-Fi, WPS",WPS button to set up wireless connection,N300,5 W


In [8]:
product.isnull().sum()

Name                       0
Price                      0
Type                       0
Model                      0
In The Box                 0
Color                      0
Brand                      0
Frequency                  0
Frequency Band             0
Wireless Speed             0
USB                        4
Antennae                   0
Number of USB Ports        0
Height                     0
Width                      0
Depth                      1
Weight                     3
Service Type               0
Covered in Warranty        0
Not Covered in Warranty    0
Warranty Summary           0
Compatible ISP             3
LAN/WAN                    1
Number of LAN ports        1
Number of WAN ports        1
Number of Antennae         1
Antennae Capacity          1
Warranty Type              2
Part Number                3
Controls                   2
Operating Humidity         2
Temperature                2
Power Supply               2
Operating System           2
Standard IEEE 